In [0]:
import os
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pickle
#mlflow
import mlflow
import mlflow.sklearn

In [0]:
import sklearn
sklearn.__version__

Out[18]: '0.22.1'

In [0]:
import mlflow
mlflow.__version__

Out[19]: '1.8.0'

In [0]:
def load_data():
  boston_dataset = load_boston()
  bdf = pd.DataFrame(boston_dataset['data'],
                     columns = boston_dataset['feature_names'])
  bdf["MEDV"] = boston_dataset['target']
  # Sanity check that it should have 13 features
  assert bdf.shape[1] == 14
  print("Asserted 13 Features and 1 target variable")
  print("printing Head of boston df")
  print(bdf.head(10))
  return bdf

In [0]:
# Data Feature selection
def feature_selection(df):
  df = df[["RM", "LSTAT", "MEDV"]]
  return df

In [0]:
def train_test(df):
  X = df[["RM", "LSTAT"]]
  y = df[["MEDV"]]
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 5)
  return X_train, X_test, y_train, y_test

In [0]:
def train_model(X_train, y_train):
  lm = LinearRegression()
  model = lm.fit(X_train, y_train)
  return model

In [0]:
# save the model to adls
# filepath ='/dbfs/mnt/inst_DataScienceProject/MSS/DEV/MSS_Workspace_Backups/linearmodel.pkl'
def save_model(filepath,model):
  pickle.dump(model, open(filepath, 'wb'))

In [0]:
def load_model(filepath):
  loaded_model =    pickle.load(open(filepath, 'rb'))
  return loaded_model

In [0]:
def predict(df, model):
  pred = model.predict(df)
  return pred

In [0]:
def eval_metrices(actual, pred):
  rmse = np.sqrt(mean_squared_error(actual, pred))
  mae = mean_absolute_error(actual, pred)
  r2 = r2_score(actual, pred)
  print("rmse = ", rmse)
  print("mae = ", mae)
  print("r2 = ", r2)
  return rmse, mae, r2

In [0]:
df = load_data()
df = feature_selection(df)

Asserted 13 Features and 1 target variable
printing Head of boston df
 CRIM ZN INDUS CHAS NOX ... TAX PTRATIO B LSTAT MEDV
0 0.00632 18.0 2.31 0.0 0.538 ... 296.0 15.3 396.90 4.98 24.0
1 0.02731 0.0 7.07 0.0 0.469 ... 242.0 17.8 396.90 9.14 21.6
2 0.02729 0.0 7.07 0.0 0.469 ... 242.0 17.8 392.83 4.03 34.7
3 0.03237 0.0 2.18 0.0 0.458 ... 222.0 18.7 394.63 2.94 33.4
4 0.06905 0.0 2.18 0.0 0.458 ... 222.0 18.7 396.90 5.33 36.2
5 0.02985 0.0 2.18 0.0 0.458 ... 222.0 18.7 394.12 5.21 28.7
6 0.08829 12.5 7.87 0.0 0.524 ... 311.0 15.2 395.60 12.43 22.9
7 0.14455 12.5 7.87 0.0 0.524 ... 311.0 15.2 396.90 19.15 27.1
8 0.21124 12.5 7.87 0.0 0.524 ... 311.0 15.2 386.63 29.93 16.5
9 0.17004 12.5 7.87 0.0 0.524 ... 311.0 15.2 386.71 17.10 18.9

[10 rows x 14 columns]

In [0]:
with mlflow.start_run():
  X_train, X_test, y_train, y_test = train_test(df)
  model = train_model(X_train, y_train)
  pred = predict(X_test, model)
  (rmse, mae, r2) = eval_metrices(y_test, pred)
  mlflow.log_metric("rmse", rmse)
  mlflow.log_metric("r2", r2)
  mlflow.log_metric("mae", mae)
  mlflow.sklearn.log_model(model, "model")
  

rmse = 5.137400784702911
mae = 3.7913102133431047
r2 = 0.6628996975186952

In [0]:
model

Out[30]: LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
# filepath ='/dbfs/mnt/inst_DataScienceProject/MSS/DEV/MSS_Workspace_Backups/linearmodel_prod.pkl'
# save_model(filepath,model)

In [0]:
pred = predict(X_test, model)

In [0]:
pred

Out[33]: array([[37.38999403],
 [29.79290611],
 [25.86755297],
 [ 0.31370828],
 [33.31385559],
 [ 7.97136102],
 [30.7066387 ],
 [27.83076842],
 [26.26693081],
 [21.97871678],
 [32.33149332],
 [23.21791374],
 [22.9932889 ],
 [30.97465356],
 [27.19747687],
 [20.7171544 ],
 [-0.67524986],
 [18.01248654],
 [12.3108109 ],
 [21.90615827],
 [ 4.82262227],
 [24.00423026],
 [37.70279396],
 [24.59521859],
 [29.6355729 ],
 [12.5396288 ],
 [27.07081337],
 [22.44485896],
 [27.64895322],
 [28.99223597],
 [11.38689571],
 [10.39119661],
 [18.02726641],
 [24.65916571],
 [26.3259374 ],
 [22.97547671],
 [26.32610451],
 [12.30204538],
 [37.03909693],
 [33.55198947],
 [20.2779501 ],
 [ 1.0905118 ],
 [27.65786778],
 [16.52789139],
 [27.49181818],
 [29.91634422],
 [-3.04746229],
 [17.23365847],
 [20.71953914],
 [13.74285813],
 [20.74965837],
 [21.48012369],
 [25.17310326],
 [16.12470269],
 [17.61200383],
 [27.89189158],
 [36.0647476 ],
 [19.67862758],
 [28.88714637],
 [20.4560256 ],
 [20.11858445],
 [23.1131674 ],
 [16.53445226],
 [31.30827991],
 [22.62162748],
 [13.10525045],
 [23.36377939],
 [25.90474345],
 [23.00735629],
 [21.62430016],
 [19.09598641],
 [26.54344302],
 [16.82687517],
 [19.99592881],
 [19.77353574],
 [30.38611207],
 [19.34927447],
 [13.03847313],
 [28.29385627],
 [19.03960282],
 [22.09279603],
 [38.79116551],
 [15.63348401],
 [20.34679092],
 [21.95060008],
 [18.33664278],
 [17.46437785],
 [ 7.42994531],
 [17.94788281],
 [24.15298313],
 [35.48406888],
 [20.19265715],
 [21.92066094],
 [19.37880824],
 [26.53637178],
 [28.83077576],
 [16.74210556],
 [30.35752395],
 [17.8896288 ],
 [15.35541079],
 [23.680157 ],
 [25.43175021]])

In [0]:
# df = spark.createDataFrame(X_test)

In [0]:
# # Prediction using pyfunc and mlflow
# import mlflow
# logged_model = 'runs:/647c7ca953034f51b74a2db3c1d3381c/model'

# # Load model as a Spark UDF.
# loaded_model = mlflow.pyfunc.spark_udf(spark, model_uri=logged_model)

# # Predict on a Spark DataFrame.
# # df.withColumn('predictions', loaded_model()).collect()
# df1 = df.withColumn('predictions', loaded_model())

In [0]:
# type(df1)

Out[80]: pyspark.sql.dataframe.DataFrame

In [0]:
# # Using pandas
# import mlflow
# logged_model = 'runs:/647c7ca953034f51b74a2db3c1d3381c/model'

# # Load model as a PyFuncModel.
# loaded_model = mlflow.pyfunc.load_model(logged_model)

# # Predict on a Pandas DataFrame.
# import pandas as pd
# loaded_model.predict(pd.DataFrame(X_test))

Out[83]: array([[37.38999403],
 [29.79290611],
 [25.86755297],
 [ 0.31370828],
 [33.31385559],
 [ 7.97136102],
 [30.7066387 ],
 [27.83076842],
 [26.26693081],
 [21.97871678],
 [32.33149332],
 [23.21791374],
 [22.9932889 ],
 [30.97465356],
 [27.19747687],
 [20.7171544 ],
 [-0.67524986],
 [18.01248654],
 [12.3108109 ],
 [21.90615827],
 [ 4.82262227],
 [24.00423026],
 [37.70279396],
 [24.59521859],
 [29.6355729 ],
 [12.5396288 ],
 [27.07081337],
 [22.44485896],
 [27.64895322],
 [28.99223597],
 [11.38689571],
 [10.39119661],
 [18.02726641],
 [24.65916571],
 [26.3259374 ],
 [22.97547671],
 [26.32610451],
 [12.30204538],
 [37.03909693],
 [33.55198947],
 [20.2779501 ],
 [ 1.0905118 ],
 [27.65786778],
 [16.52789139],
 [27.49181818],
 [29.91634422],
 [-3.04746229],
 [17.23365847],
 [20.71953914],
 [13.74285813],
 [20.74965837],
 [21.48012369],
 [25.17310326],
 [16.12470269],
 [17.61200383],
 [27.89189158],
 [36.0647476 ],
 [19.67862758],
 [28.88714637],
 [20.4560256 ],
 [20.11858445],
 [23.1131674 ],
 [16.53445226],
 [31.30827991],
 [22.62162748],
 [13.10525045],
 [23.36377939],
 [25.90474345],
 [23.00735629],
 [21.62430016],
 [19.09598641],
 [26.54344302],
 [16.82687517],
 [19.99592881],
 [19.77353574],
 [30.38611207],
 [19.34927447],
 [13.03847313],
 [28.29385627],
 [19.03960282],
 [22.09279603],
 [38.79116551],
 [15.63348401],
 [20.34679092],
 [21.95060008],
 [18.33664278],
 [17.46437785],
 [ 7.42994531],
 [17.94788281],
 [24.15298313],
 [35.48406888],
 [20.19265715],
 [21.92066094],
 [19.37880824],
 [26.53637178],
 [28.83077576],
 [16.74210556],
 [30.35752395],
 [17.8896288 ],
 [15.35541079],
 [23.680157 ],
 [25.43175021]])

In [0]:
%sh 
databricks fs -h

Usage: databricks fs [OPTIONS] COMMAND [ARGS]...

 Utility to interact with DBFS.

 DBFS paths are all prefixed with dbfs:/. Local paths can be absolute or
 local.

Options:
 -v, --version 0.12.2
 --debug Debug Mode. Shows full stack trace on error.
 --profile TEXT CLI connection profile to use. The default profile is
 "DEFAULT".

 -h, --help Show this message and exit.

Commands:
 cat Show the contents of a file.
 configure Configures host and authentication info for the CLI.
 cp Copy files to and from DBFS.
 ls List files in DBFS.
 mkdirs Make directories in DBFS.
 mv Moves a file between two DBFS paths.
 rm Remove files from dbfs.

In [0]:
%sh ls /loac

BUILD
bin
boot
databricks
dbfs
dev
etc
home
lib
lib64
local_disk0
media
mnt
opt
proc
root
run
sbin
srv
sys
tmp
usr
var

In [0]:
%sh
databricks fs cp "dbfs:/databricks/mlflow-tracking/2376261929218913/647c7ca953034f51b74a2db3c1d3381c/artifacts/model/" "/local_disk0/tmp/jay/"

Error: b'{"error_code":"PERMISSION_DENIED","message":"No operations allowed on this path"}'

In [0]:
# # Loading model from artifacts
# lm = mlflow.sklearn.load_model("dbfs:/databricks/mlflow-tracking/2376261929218913/647c7ca953034f51b74a2db3c1d3381c/artifacts/model/")


In [0]:
# lm.predict(X_test)

Out[114]: array([[37.38999403],
 [29.79290611],
 [25.86755297],
 [ 0.31370828],
 [33.31385559],
 [ 7.97136102],
 [30.7066387 ],
 [27.83076842],
 [26.26693081],
 [21.97871678],
 [32.33149332],
 [23.21791374],
 [22.9932889 ],
 [30.97465356],
 [27.19747687],
 [20.7171544 ],
 [-0.67524986],
 [18.01248654],
 [12.3108109 ],
 [21.90615827],
 [ 4.82262227],
 [24.00423026],
 [37.70279396],
 [24.59521859],
 [29.6355729 ],
 [12.5396288 ],
 [27.07081337],
 [22.44485896],
 [27.64895322],
 [28.99223597],
 [11.38689571],
 [10.39119661],
 [18.02726641],
 [24.65916571],
 [26.3259374 ],
 [22.97547671],
 [26.32610451],
 [12.30204538],
 [37.03909693],
 [33.55198947],
 [20.2779501 ],
 [ 1.0905118 ],
 [27.65786778],
 [16.52789139],
 [27.49181818],
 [29.91634422],
 [-3.04746229],
 [17.23365847],
 [20.71953914],
 [13.74285813],
 [20.74965837],
 [21.48012369],
 [25.17310326],
 [16.12470269],
 [17.61200383],
 [27.89189158],
 [36.0647476 ],
 [19.67862758],
 [28.88714637],
 [20.4560256 ],
 [20.11858445],
 [23.1131674 ],
 [16.53445226],
 [31.30827991],
 [22.62162748],
 [13.10525045],
 [23.36377939],
 [25.90474345],
 [23.00735629],
 [21.62430016],
 [19.09598641],
 [26.54344302],
 [16.82687517],
 [19.99592881],
 [19.77353574],
 [30.38611207],
 [19.34927447],
 [13.03847313],
 [28.29385627],
 [19.03960282],
 [22.09279603],
 [38.79116551],
 [15.63348401],
 [20.34679092],
 [21.95060008],
 [18.33664278],
 [17.46437785],
 [ 7.42994531],
 [17.94788281],
 [24.15298313],
 [35.48406888],
 [20.19265715],
 [21.92066094],
 [19.37880824],
 [26.53637178],
 [28.83077576],
 [16.74210556],
 [30.35752395],
 [17.8896288 ],
 [15.35541079],
 [23.680157 ],
 [25.43175021]])

In [0]:
# import mlflow
# p_uri = "dbfs:/databricks/mlflow-tracking/2376261929218913/3b3e5bc8425d4635afe3816d68f2f313/artifacts/model"
# mlflow.run(p_uri)

--------------------------------------------------------------------------- 
 GitCommandError Traceback (most recent call last)
 <command-2160686963457181> in <module> 
 1 import mlflow
 2 p_uri = "dbfs:/databricks/mlflow-tracking/2376261929218913/3b3e5bc8425d4635afe3816d68f2f313/artifacts/model" 
 ----> 3 mlflow . run ( p_uri ) 

 /databricks/python/lib/python3.7/site-packages/mlflow/projects/__init__.py in run (uri, entry_point, version, parameters, docker_args, experiment_name, experiment_id, backend, backend_config, use_conda, storage_dir, synchronous, run_id) 
 289 parameters = parameters , docker_args = docker_args , backend = backend , 
 290 backend_config = cluster_spec_dict , use_conda = use_conda , storage_dir = storage_dir , 
 --> 291 synchronous=synchronous, run_id=run_id)
 292 if synchronous : 
 293 _wait_for ( submitted_run_obj ) 

 /databricks/python/lib/python3.7/site-packages/mlflow/projects/__init__.py in _run (uri, experiment_id, entry_point, version, parameters, docker_args, backend, backend_config, use_conda, storage_dir, synchronous, run_id) 
 103 
 104 parameters = parameters or { } 
 --> 105 work_dir = _fetch_project ( uri = uri , force_tempdir = False , version = version ) 
 106 project = _project_spec . load_project ( work_dir ) 
 107 _validate_execution_environment ( project , backend ) 

 /databricks/python/lib/python3.7/site-packages/mlflow/projects/__init__.py in _fetch_project (uri, force_tempdir, version) 
 343 else : 
 344 assert _GIT_URI_REGEX . match ( parsed_uri ) , "Non-local URI %s should be a Git URI" % parsed_uri
 --> 345 _fetch_git_repo ( parsed_uri , version , dst_dir ) 
 346 res = os . path . abspath ( os . path . join ( dst_dir , subdirectory ) ) 
 347 if not os . path . exists ( res ) : 

 /databricks/python/lib/python3.7/site-packages/mlflow/projects/__init__.py in _fetch_git_repo (uri, version, dst_dir) 
 383 repo = git . Repo . init ( dst_dir ) 
 384 origin = repo . create_remote ( "origin" , uri ) 
 --> 385 origin . fetch ( ) 
 386 if version is not None : 
 387 try : 

 /databricks/python/lib/python3.7/site-packages/git/remote.py in fetch (self, refspec, progress, **kwargs) 
 787 proc = self.repo.git.fetch(self, *args, as_process=True, with_stdout=False,
 788 universal_newlines=True, v=True, **kwargs)
 --> 789 res = self . _get_fetch_info_from_stderr ( proc , progress ) 
 790 if hasattr ( self . repo . odb , 'update_cache' ) : 
 791 self . repo . odb . update_cache ( ) 

 /databricks/python/lib/python3.7/site-packages/git/remote.py in _get_fetch_info_from_stderr (self, proc, progress) 
 673 
 674 stderr_text = progress . error_lines and '\n' . join ( progress . error_lines ) or '' 
 --> 675 proc . wait ( stderr = stderr_text ) 
 676 if stderr_text : 
 677 log . warning ( "Error lines received while fetching: %s" , stderr_text ) 

 /databricks/python/lib/python3.7/site-packages/git/cmd.py in wait (self, stderr) 
 413 errstr = read_all_from_possibly_closed_stream ( self . proc . stderr ) 
 414 log . debug ( 'AutoInterrupt wait stderr: %r' % ( errstr , ) ) 
 --> 415 raise GitCommandError ( self . args , status , errstr ) 
 416 # END status handling 
 417 return status

 GitCommandError : Cmd('git') failed due to: exit code(128)
 cmdline: git fetch -v origin
 stderr: 'fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.'